In [18]:
import numpy 
import cupy
from humanize import naturalsize

use_gpus = [0, 1]

for use_gpu in [ 0, 1 ] : 
    print( f"use_gpu = {use_gpu}" )
    print()

    np = cupy if use_gpu else numpy

    data_types = [ np.int_, np.longlong, np.double, np.csingle, np.cdouble ]

    for data_type in data_types :
        type_str = f"{data_type}".split( " " )[-1].split(".")[-1].split( "'")[0]
        type_str = [ "numpy ", "cupy "][use_gpu] + type_str
        
        print( type_str )
        
        for res in range( 10, 21 , 1 ) :
            try : 
                resolution = 2**res
                
                array_1 = np.zeros( (resolution, resolution ) , data_type )
                array_2 = np.zeros( (resolution, resolution ) , data_type )
                array_3 = array_1*array_2 
                sum = np.sum( array_3 )
                
                size = 0 ; size_m = 0 
                for array in [ array_1, array_2, array_3 ] : 
                    size += array.nbytes
                    size_m += size / 10**6
                pass
                print( f"resolution = {resolution:_}, size = {size}, size_m = {size_m:_.1f}, natural size = {naturalsize(size)}" )
                del array
            except  : 
                break;
            pass
        pass
    
        print()
    pass

    print()
pass

print( "Done." )

use_gpu = 0

numpy int32
resolution = 1_024, size = 12582912, size_m = 25.2, natural size = 12.6 MB
resolution = 2_048, size = 50331648, size_m = 100.7, natural size = 50.3 MB
resolution = 4_096, size = 201326592, size_m = 402.7, natural size = 201.3 MB
resolution = 8_192, size = 805306368, size_m = 1_610.6, natural size = 805.3 MB
resolution = 16_384, size = 3221225472, size_m = 6_442.5, natural size = 3.2 GB
resolution = 32_768, size = 12884901888, size_m = 25_769.8, natural size = 12.9 GB

numpy int64
resolution = 1_024, size = 25165824, size_m = 50.3, natural size = 25.2 MB
resolution = 2_048, size = 100663296, size_m = 201.3, natural size = 100.7 MB
resolution = 4_096, size = 402653184, size_m = 805.3, natural size = 402.7 MB
resolution = 8_192, size = 1610612736, size_m = 3_221.2, natural size = 1.6 GB
resolution = 16_384, size = 6442450944, size_m = 12_884.9, natural size = 6.4 GB
resolution = 32_768, size = 25769803776, size_m = 51_539.6, natural size = 25.8 GB

numpy float64
r